# Testing 5-bit trigger codes w/ arduino & pyserial

In [8]:
import serial
import time

### CONFIG

In [3]:
# CHANGE THIS to match your Arduino port:
#   Windows: "COM3", "COM4", ...
#   macOS:   "/dev/tty.usbmodemXXXX" or "/dev/tty.usbserialXXXX"
#   Linux:   "/dev/ttyACM0", "/dev/ttyUSB0", etc.
SERIAL_PORT = "/dev/ttyACM0"
BAUD_RATE = 115200


## Functions

In [9]:

def open_trigger_serial(port=SERIAL_PORT, baud_rate=BAUD_RATE):
    ser = serial.Serial(port, baud_rate, timeout=1)
    # Arduino auto-resets when serial opens; give it a moment
    time.sleep(2.0)
    
    print(f"Opened serial port {port} at {baud_rate} baud.")
    
    return ser

def send_trigger(ser, code):
    """
    Send a trigger code in [0, 31] to Arduino.
    Arduino maps the 5 LSBs to pins (4, 6, 8, 10, 12)
    and pulses them for PULSE_MS ms.
    """
    if not (0 <= code <= 31):
        raise ValueError("Trigger code must be between 0 and 31 (inclusive).")

    ser.write(bytes([code]))
    ser.flush()   # push out immediately

## Tests

In [10]:
ser = open_trigger_serial()

try:
    # Quick test: step through some codes
    for code in [1, 2, 4, 8, 16, 3, 7, 15, 31]:
        print(f"Sending trigger code: {code}")
        send_trigger(ser, code)
        time.sleep(0.5)
finally:
    ser.close()

Opened serial port /dev/ttyACM0 at 115200 baud.
Sending trigger code: 1
Sending trigger code: 2
Sending trigger code: 4
Sending trigger code: 8
Sending trigger code: 16
Sending trigger code: 3
Sending trigger code: 7
Sending trigger code: 15
Sending trigger code: 31
